# QF609 Risk Analysis
## Group Assignment I 
## VaR (Value at Risk)
YU Lingfeng

In [1]:
# importing labraries
import numpy as np
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.optimize import brentq, curve_fit, least_squares
from scipy.stats import norm
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

## <a id = "top">Table of Content</a>
### [Part 1. 10-Yr Swap Payer Risk Factors](#p1)   
$PV_{fix} = S_{0} \cdot \Delta_{1y} \cdot (D(0, 1y)+ D(0, 2y)+D(0, 3y) + ... + D(0, 10y)) + D(0,10y)$

$PV_{float} =1$

$V^{pay} = PV_{float}-PV_{fix}$

### [Part 2. Stocks Risk Factors](#p2)   

### [Part 3. Parametric VaR Model](#p3)
$\mu_P = \omega \cdot \mu$

$\sigma^2_P = \omega \cdot \Sigma \cdot {\omega}^T $

### [Part 4. Monte Carlo VaR Model](#p4)

- ##### [4.1 Full Revaluation](#p41)
- ##### [4.2 Risk-based](#p42)

### [Part 5. Historical VaR Model](#p5)

- ##### [5.1 Full Revaluation](#p51)
- ##### [5.2 Risk-based](#p52)

In [2]:
sofr =\
    pd.read_excel('hist_data.xlsm',
                  sheet_name='SofrCurve', 
                  index_col = [0],
                  header=[0])
print(sofr.shape)
sofr.head()

(30, 252)


,T,2022-10-31 00:00:00,2022-11-01 00:00:00,2022-11-02 00:00:00,2022-11-03 00:00:00,2022-11-04 00:00:00,2022-11-07 00:00:00,2022-11-08 00:00:00,2022-11-09 00:00:00,2022-11-10 00:00:00,...,2023-10-17 00:00:00,2023-10-18 00:00:00,2023-10-19 00:00:00,2023-10-20 00:00:00,2023-10-23 00:00:00,2023-10-24 00:00:00,2023-10-25 00:00:00,2023-10-26 00:00:00,2023-10-27 00:00:00,2023-10-30 00:00:00
Tenor,,,,,,,,,,,,,,,,,,,,,
1D,0.002778,0.039191,0.039604,0.039948,0.040389,0.045965,0.040224,0.040339,0.040373,0.052075,...,0.052964,0.053038,0.053050,0.053072,0.053090,0.053105,0.052981,0.053047,0.052989,0.052967
1M,0.083333,0.038721,0.039023,0.039286,0.039585,0.042343,0.039258,0.039211,0.039193,0.043694,...,0.053114,0.053102,0.053086,0.053046,0.053077,0.053084,0.053057,0.053089,0.053040,0.053053
2M,0.166667,0.038670,0.038886,0.039100,0.039350,0.038795,0.039460,0.039420,0.039483,0.038786,...,0.053374,0.053284,0.053226,0.053128,0.053191,0.053202,0.053259,0.053218,0.053185,0.053265
3M,0.250000,0.040536,0.040725,0.040852,0.041154,0.040611,0.041424,0.041406,0.041445,0.040867,...,0.053666,0.053561,0.053454,0.053323,0.053404,0.053424,0.053490,0.053414,0.053368,0.053476
6M,0.500000,0.044577,0.044849,0.044884,0.045281,0.045212,0.045455,0.045334,0.045299,0.044951,...,0.054078,0.053989,0.053758,0.053546,0.053638,0.053690,0.053749,0.053567,0.053486,0.053612


In [3]:
aapl =\
    pd.read_excel('hist_data.xlsm',
                  sheet_name='AAPL', 
                  index_col = 0,
                  header=[0])
print(aapl.shape)
aapl.head()

(251, 1)


,Adj Close
Date,
2022-10-31,152.041122
2022-11-01,149.373917
2022-11-02,143.801514
2022-11-03,137.703613
2022-11-04,137.435455


In [4]:
msft =\
    pd.read_excel('hist_data.xlsm',
                  sheet_name='MSFT', 
                  index_col = 0,
                  header=[0])
print(msft.shape)
msft.head()

(251, 1)


,Adj Close
Date,
2022-10-31,229.443207
2022-11-01,225.529037
2022-11-02,217.552444
2022-11-03,211.770157
2022-11-04,218.827515


In [5]:
f =\
    pd.read_excel('hist_data.xlsm',
                  sheet_name='F', 
                  index_col = 0,
                  header=[0])
print(f.shape)
f.head()

(251, 1)


,Adj Close
Date,
2022-10-31,11.974380
2022-11-01,12.001248
2022-11-02,11.696740
2022-11-03,11.875863
2022-11-04,12.099767


In [6]:
bac =\
    pd.read_excel('hist_data.xlsm',
                  sheet_name='BAC', 
                  index_col = 0,
                  header=[0])
print(bac.shape)
bac.head()

(251, 1)


,Adj Close
Date,
2022-10-31,34.748581
2022-11-01,34.902847
2022-11-02,34.796783
2022-11-03,34.603954
2022-11-04,35.471703


In [7]:
sofr.index

Index(['1D', '1M', '2M', '3M', '6M', '9M', '1Y', '2Y', '3Y', '4Y', '5Y', '6Y',
       '7Y', '8Y', '9Y', '10Y', '11Y', '12Y', '13Y', '14Y', '15Y', '16Y',
       '17Y', '18Y', '19Y', '20Y', '25Y', '30Y', '35Y', '40Y'],
      dtype='object', name='Tenor')

In [8]:
r_factors =\
    sofr.loc[(sofr['T']>=1) & (sofr['T']<=10)]
Tenor_list = list(r_factors.index)

r_factors =\
    r_factors\
    .set_index('T')\
    .T
T_list = list(r_factors.columns)
r_factors

T,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
2022-10-31,0.046449,0.044583,0.042002,0.040318,0.039297,0.038534,0.037979,0.037648,0.037465,0.037353
2022-11-01,0.046970,0.045022,0.042344,0.040614,0.039522,0.038658,0.038007,0.037605,0.037368,0.037209
2022-11-02,0.047203,0.045496,0.042749,0.040868,0.039675,0.038776,0.038117,0.037704,0.037458,0.037300
2022-11-03,0.047894,0.046594,0.043833,0.041825,0.040507,0.039537,0.038820,0.038328,0.038002,0.037796
2022-11-04,0.047500,0.046097,0.043385,0.041503,0.040303,0.039421,0.038782,0.038373,0.038128,0.037988
...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.052503,0.048399,0.045999,0.044850,0.044305,0.044036,0.043889,0.043816,0.043798,0.043826
2023-10-25,0.052653,0.048791,0.046595,0.045594,0.045200,0.045017,0.044911,0.044867,0.044876,0.044926
2023-10-26,0.052243,0.048044,0.045645,0.044538,0.044086,0.043893,0.043808,0.043795,0.043829,0.043898
2023-10-27,0.052115,0.047758,0.045284,0.044200,0.043762,0.043636,0.043643,0.043702,0.043793,0.043908


In [9]:
Tenor_list

['1Y', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y']

In [10]:
Stock_list = ['AAPL', 'MSFT', 'F', 'BAC']
s_factors = pd.concat([aapl, msft, f, bac], axis = 1)
s_factors.columns = Stock_list
s_factors

,AAPL,MSFT,F,BAC
Date,,,,
2022-10-31,152.041122,229.443207,11.974380,34.748581
2022-11-01,149.373917,225.529037,12.001248,34.902847
2022-11-02,143.801514,217.552444,11.696740,34.796783
2022-11-03,137.703613,211.770157,11.875863,34.603954
2022-11-04,137.435455,218.827515,12.099767,35.471703
...,...,...,...,...
2023-10-24,172.991058,329.860504,11.215128,25.268324
2023-10-25,170.657135,339.979980,11.362824,25.347691
2023-10-26,166.458023,327.225861,11.175742,25.913177


In [11]:
P_factors =\
    pd.concat([r_factors, 
               s_factors], 
              axis = 1)\
    .sort_index()\
    .interpolate()
print(P_factors.shape)
P_factors

(253, 14)


,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,AAPL,MSFT,F,BAC
2022-10-31,0.046449,0.044583,0.042002,0.040318,0.039297,0.038534,0.037979,0.037648,0.037465,0.037353,152.041122,229.443207,11.974380,34.748581
2022-11-01,0.046970,0.045022,0.042344,0.040614,0.039522,0.038658,0.038007,0.037605,0.037368,0.037209,149.373917,225.529037,12.001248,34.902847
2022-11-02,0.047203,0.045496,0.042749,0.040868,0.039675,0.038776,0.038117,0.037704,0.037458,0.037300,143.801514,217.552444,11.696740,34.796783
2022-11-03,0.047894,0.046594,0.043833,0.041825,0.040507,0.039537,0.038820,0.038328,0.038002,0.037796,137.703613,211.770157,11.875863,34.603954
2022-11-04,0.047500,0.046097,0.043385,0.041503,0.040303,0.039421,0.038782,0.038373,0.038128,0.037988,137.435455,218.827515,12.099767,35.471703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.052503,0.048399,0.045999,0.044850,0.044305,0.044036,0.043889,0.043816,0.043798,0.043826,172.991058,329.860504,11.215128,25.268324
2023-10-25,0.052653,0.048791,0.046595,0.045594,0.045200,0.045017,0.044911,0.044867,0.044876,0.044926,170.657135,339.979980,11.362824,25.347691
2023-10-26,0.052243,0.048044,0.045645,0.044538,0.044086,0.043893,0.043808,0.043795,0.043829,0.043898,166.458023,327.225861,11.175742,25.913177
2023-10-27,0.052115,0.047758,0.045284,0.044200,0.043762,0.043636,0.043643,0.043702,0.043793,0.043908,167.784576,329.141968,9.807083,24.970699


## <a id = "p1"> Q1. </a>  10-Yr Swap Payer Risk Factors [back to table of contents](#top)

In [12]:
unit_pip = 1e-4
r_risk =\
    P_factors[T_list]\
    .diff()/\
    unit_pip
r_risk

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
2022-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-01,5.209359,4.389513,3.418913,2.963925,2.252118,1.239837,0.286314,-0.429620,-0.972444,-1.437657
2022-11-02,2.337597,4.739896,4.044957,2.540902,1.527523,1.181667,1.091212,0.986698,0.899499,0.914312
2022-11-03,6.902891,10.979397,10.846471,9.568749,8.321454,7.607816,7.037746,6.239340,5.446595,4.953354
2022-11-04,-3.941057,-4.966621,-4.479350,-3.224663,-2.039849,-1.155399,-0.378675,0.454820,1.258218,1.921468
...,...,...,...,...,...,...,...,...,...,...
2023-10-24,1.141627,3.713939,4.166370,3.425024,2.574721,1.647722,0.831521,0.321099,0.002758,-0.272156
2023-10-25,1.501560,3.926395,5.958683,7.432668,8.943427,9.812561,10.217192,10.512852,10.774863,11.001288
2023-10-26,-4.100263,-7.472812,-9.497604,-10.551848,-11.139795,-11.242255,-11.020763,-10.726409,-10.462316,-10.279028
2023-10-27,-1.288150,-2.863713,-3.607100,-3.385875,-3.239752,-2.568867,-1.658365,-0.926249,-0.366780,0.105042


## <a id = "p2"> Q2. </a>  Stock Portfolio Risk Factors [back to table of contents](#top)

In [13]:
s_risk =\
    P_factors[Stock_list]\
    .pct_change()
s_risk

,AAPL,MSFT,F,BAC
2022-10-31,NaN,NaN,NaN,NaN
2022-11-01,-0.017543,-0.017059,0.002244,0.004439
2022-11-02,-0.037305,-0.035368,-0.025373,-0.003039
2022-11-03,-0.042405,-0.026579,0.015314,-0.005542
2022-11-04,-0.001947,0.033326,0.018854,0.025077
...,...,...,...,...
2023-10-24,0.002543,0.003674,-0.007840,-0.003911
2023-10-25,-0.013492,0.030678,0.013169,0.003141
2023-10-26,-0.024606,-0.037514,-0.016464,0.022309
2023-10-27,0.007969,0.005856,-0.122467,-0.036371


In [14]:
P_risk =\
    pd.concat([r_risk, s_risk], axis = 1).dropna()
print(P_risk.shape)
P_risk

(252, 14)


,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,AAPL,MSFT,F,BAC
2022-11-01,5.209359,4.389513,3.418913,2.963925,2.252118,1.239837,0.286314,-0.429620,-0.972444,-1.437657,-0.017543,-0.017059,0.002244,0.004439
2022-11-02,2.337597,4.739896,4.044957,2.540902,1.527523,1.181667,1.091212,0.986698,0.899499,0.914312,-0.037305,-0.035368,-0.025373,-0.003039
2022-11-03,6.902891,10.979397,10.846471,9.568749,8.321454,7.607816,7.037746,6.239340,5.446595,4.953354,-0.042405,-0.026579,0.015314,-0.005542
2022-11-04,-3.941057,-4.966621,-4.479350,-3.224663,-2.039849,-1.155399,-0.378675,0.454820,1.258218,1.921468,-0.001947,0.033326,0.018854,0.025077
2022-11-07,4.292194,5.361705,4.990763,4.128393,3.318854,3.127938,3.283606,3.417098,3.452884,3.393246,0.003902,0.029270,0.014064,0.005980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,1.141627,3.713939,4.166370,3.425024,2.574721,1.647722,0.831521,0.321099,0.002758,-0.272156,0.002543,0.003674,-0.007840,-0.003911
2023-10-25,1.501560,3.926395,5.958683,7.432668,8.943427,9.812561,10.217192,10.512852,10.774863,11.001288,-0.013492,0.030678,0.013169,0.003141
2023-10-26,-4.100263,-7.472812,-9.497604,-10.551848,-11.139795,-11.242255,-11.020763,-10.726409,-10.462316,-10.279028,-0.024606,-0.037514,-0.016464,0.022309
2023-10-27,-1.288150,-2.863713,-3.607100,-3.385875,-3.239752,-2.568867,-1.658365,-0.926249,-0.366780,0.105042,0.007969,0.005856,-0.122467,-0.036371


## <a id = "p3"> Q3. </a>  Parametric VaR Model [back to table of contents](#top)

In [15]:
S = 0.042
N = 1e8
stock_w = [1e6] * 4

In [16]:
def V_pay(DF, S, N):
    fix = sum(DF)*S
    flt = 1-DF[-1]
    return N*(flt-fix)

In [17]:
r_zeros =\
    pd.DataFrame(r_factors.T.iloc[:,-1])\
    .reset_index()
r_zeros.index = Tenor_list
r_zeros.columns =\
    ['T', 'r']
r_zeros['DF'] =\
    np.exp(-r_zeros['r'] * r_zeros['T'])
r_zeros['PV01'] = 0
unit_pip = 1e-4
for i in range(len(r_zeros)):
    r_temp = r_zeros.copy()['r']
    r_temp[i] += unit_pip
    DF_temp =\
        np.exp(-r_temp * r_zeros['T'])
    r_zeros.iloc[i,-1] =\
        V_pay(DF_temp, S, N) -\
        V_pay(r_zeros['DF'], S, N)  
r_zeros

,T,r,DF,PV01
1Y,1.0,0.052245,0.949097,398.600677
2Y,2.0,0.047904,0.908638,763.179678
3Y,3.0,0.045429,0.872591,1099.300108
4Y,4.0,0.044345,0.837462,1406.655357
5Y,5.0,0.043928,0.802808,1685.475099
6Y,6.0,0.043794,0.768923,1937.105609
7Y,7.0,0.043779,0.736051,2163.233594
8Y,8.0,0.043828,0.704247,2365.322757
9Y,9.0,0.043915,0.673524,2544.776906
10Y,10.0,0.044023,0.643887,67059.526256


In [18]:
V_0 =\
    V_pay(r_zeros['DF'], S, N)
V_0

2442901.9998463197

$\mu_P = \omega \cdot \mu$

$\sigma^2_P = \omega \cdot \Sigma \cdot {\omega}^T $

In [19]:
swap_w = list(r_zeros['PV01'])
Para_w = np.array(swap_w + stock_w)
Para_w

array([3.98600677e+02, 7.63179678e+02, 1.09930011e+03, 1.40665536e+03,
       1.68547510e+03, 1.93710561e+03, 2.16323359e+03, 2.36532276e+03,
       2.54477691e+03, 6.70595263e+04, 1.00000000e+06, 1.00000000e+06,
       1.00000000e+06, 1.00000000e+06])

In [20]:
Sigma =\
    P_risk\
    .cov()\
    .to_numpy()
print(Sigma.shape)
Sigma

(14, 14)


array([[ 6.36710634e+01,  6.83450401e+01,  6.04698591e+01,
         5.48612963e+01,  4.95908446e+01,  4.52644577e+01,
         4.20506075e+01,  3.96189713e+01,  3.75835212e+01,
         3.56371585e+01, -2.56345292e-03, -9.99449224e-03,
         1.83625101e-02,  4.51239281e-02],
       [ 6.83450401e+01,  8.27419600e+01,  7.81159713e+01,
         7.28205208e+01,  6.72447004e+01,  6.23893664e+01,
         5.85193606e+01,  5.53806041e+01,  5.26571128e+01,
         5.01123540e+01, -8.33117907e-03, -1.45582710e-02,
         1.50118878e-02,  4.54736997e-02],
       [ 6.04698591e+01,  7.81159713e+01,  7.69138203e+01,
         7.32243962e+01,  6.87928478e+01,  6.47122496e+01,
         6.12620622e+01,  5.83192046e+01,  5.56902863e+01,
         5.32379164e+01, -1.10294414e-02, -1.49994252e-02,
         9.37270491e-03,  3.73514244e-02],
       [ 5.48612963e+01,  7.28205208e+01,  7.32243962e+01,
         7.10525851e+01,  6.75895877e+01,  6.41916835e+01,
         6.12474534e+01,  5.86387787e+01,  5.

In [21]:
Mu = P_risk.mean().to_numpy()
Mu

array([ 0.22999275,  0.13179824,  0.13599857,  0.15980025,  0.18377354,
        0.20872886,  0.23017924,  0.24525335,  0.25594221,  0.26470792,
        0.00056323,  0.00167856, -0.00061515, -0.00109646])

In [22]:
Para_mean = Para_w @ Mu.T
Para_var =\
    Para_w @ Sigma @ Para_w.T
print ('portfolio Mean: ', Para_mean)
print ('portfolio Variance: ', Para_var)

portfolio Mean:  21291.345871637397
portfolio Variance:  332092585077.91626


In [23]:
# Find Z-value for 5% (lower tail)
percentile = 5
VaR_5 =\
    abs(
    Para_mean +\
    np.sqrt(Para_var) *\
    norm.ppf(percentile / 100)
    )

print("Portfolio 1d VaR at 5% is ", VaR_5)

Portfolio 1d VaR at 5% is  926596.2633475048


In [24]:
Base_V = V_0 + sum(stock_w)
print('Portfolio Base Value is: ', Base_V)

Portfolio Base Value is:  6442901.99984632


## <a id = "p4"> Q4. </a>  Monte Carlo VaR Model [back to table of contents](#top)

In [25]:
def VaR_percentile(SR, percentile_N):
    idx =\
        percentile_N/100 *\
        len(SR) - 1
    lower = int(idx)
    upper = lower + 1
    
    VAR =\
        SR[lower] +\
        (SR[upper] - SR[lower]) *\
        (idx - lower) * 1
    if VAR > 0:
        VAR = 0    
    return abs(VAR)

In [26]:
np.random.seed(100)
num_samples = 30000
samples =\
    np.random.multivariate_normal(Mu, Sigma, num_samples)
print(samples.shape)
samples=pd.DataFrame(samples)
samples

(30000, 14)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,12.093521,13.665913,13.391341,13.991770,14.012610,13.745164,13.605090,13.684660,13.770423,13.650372,-0.003145,0.009369,-0.007601,0.011190
1,-4.800231,-4.852643,-4.432978,-5.528730,-5.268043,-5.043384,-5.090500,-5.045265,-4.846567,-4.460616,0.005721,-0.003622,-0.000534,0.005393
2,2.803343,2.338613,2.128899,2.703976,3.209873,3.277242,3.123350,2.802321,2.438878,2.151843,-0.006142,0.004925,-0.030401,-0.033945
3,5.186736,0.786453,-0.982129,-0.498077,-0.679482,-0.426079,-0.007548,-0.124406,-0.485813,-0.664889,0.027155,0.027978,0.018718,-0.000696
4,0.713274,3.613157,5.874139,7.130917,8.069581,8.634343,8.856820,8.688735,8.368755,8.128647,0.019034,0.006936,0.029012,0.017180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.717646,-1.203048,-2.317071,-3.403425,-4.431892,-5.026816,-5.164341,-5.044597,-4.859408,-4.770000,-0.013102,-0.003558,0.024363,-0.003234
29996,0.246431,-0.200502,-0.214299,-0.147935,0.463872,0.560064,0.446492,0.686272,1.081772,1.335692,0.005944,0.016250,-0.008629,0.021646
29997,-12.670574,-16.177799,-16.367517,-15.085240,-13.456109,-12.293875,-11.509867,-10.728681,-9.926544,-9.178086,0.003292,0.000654,-0.022135,-0.007079
29998,-0.034299,-0.606876,-0.380123,-0.183679,-0.015852,0.183900,0.351314,0.486390,0.594688,0.689262,-0.005827,-0.016420,-0.025393,-0.023789


##### <a id = "p41"> </a> 4.1  Monte Carlo - Full Revaluation [back to table of contents](#top)

In [27]:
P_factors.iloc[-1,:]

1.0       0.052245
2.0       0.047904
3.0       0.045429
4.0       0.044345
5.0       0.043928
6.0       0.043794
7.0       0.043779
8.0       0.043828
9.0       0.043915
10.0      0.044023
AAPL    169.849197
MSFT    336.626770
F         9.620000
BAC      25.486582
Name: 2023-10-30 00:00:00, dtype: float64

In [28]:
r_samples_full =\
    samples.iloc[:, :len(Tenor_list)] *\
    unit_pip +\
    (P_factors.iloc[-1,:len(Tenor_list)]).values

s_samples_full =\
    (samples.iloc[:, -len(stock_w):]+1) *\
    (P_factors.iloc[-1,-len(stock_w):]).values

samples_full =\
    pd.concat([r_samples_full, s_samples_full],
              axis = 1)
samples_full

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.053454,0.049271,0.046768,0.045744,0.045329,0.045169,0.045140,0.045197,0.045292,0.045388,169.314964,339.780758,9.546883,25.771778
1,0.051765,0.047419,0.044986,0.043792,0.043401,0.043290,0.043270,0.043324,0.043430,0.043577,170.820917,335.407464,9.614860,25.624033
2,0.052525,0.048138,0.045642,0.044615,0.044249,0.044122,0.044092,0.044109,0.044158,0.044238,168.806009,338.284806,9.327544,24.621436
3,0.052763,0.047983,0.045331,0.044295,0.043860,0.043751,0.043779,0.043816,0.043866,0.043957,174.461434,346.045053,9.800067,25.468838
4,0.052316,0.048266,0.046017,0.045058,0.044735,0.044657,0.044665,0.044697,0.044751,0.044836,173.082169,338.961622,9.899097,25.924439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.052316,0.047784,0.045198,0.044004,0.043485,0.043291,0.043263,0.043324,0.043429,0.043546,167.623811,335.429195,9.854369,25.404156
29996,0.052269,0.047884,0.045408,0.044330,0.043974,0.043850,0.043824,0.043897,0.044023,0.044157,170.858811,342.097094,9.536988,26.038259
29997,0.050978,0.046286,0.043793,0.042836,0.042582,0.042565,0.042628,0.042755,0.042922,0.043105,170.408291,336.846941,9.407064,25.306168
29998,0.052241,0.047844,0.045391,0.044326,0.043926,0.043812,0.043814,0.043877,0.043974,0.044092,168.859525,331.099454,9.375720,24.880270


In [29]:
Base_V

6442901.99984632

In [30]:
T_list

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

In [31]:
stock_units =\
    stock_w/P_factors.iloc[-1,-len(stock_w):]
stock_units

AAPL      5887.575671
MSFT      2970.649066
F       103950.103950
BAC      39236.332279
Name: 2023-10-30 00:00:00, dtype: float64

In [32]:
DeltaV_MC_full = []

for i in range(len(samples_full)):
    DF =\
        list(np.exp(-samples_full\
                    .iloc[i, :len(T_list)] *\
                    T_list))
    V_payer =\
        V_pay(DF, S, N)
    V_stock =\
        np.array(stock_units) @\
        samples_full.iloc[i, -len(stock_units):]

    DeltaV_MC_full +=\
        [V_payer+V_stock-Base_V]
    
DeltaV_MC_full =\
    pd.DataFrame(DeltaV_MC_full)
DeltaV_MC_full.columns =\
    ['DeltaV_MC_full']
DeltaV_MC_full

,DeltaV_MC_full
0,1.115409e+06
1,-3.653154e+05
2,1.190289e+05
3,2.589793e+04
4,7.259259e+05
...,...
29995,-3.770925e+05
29996,1.314778e+05
29997,-8.231992e+05
29998,-2.259926e+04


In [33]:
DeltaV_MC_full_sort =\
    DeltaV_MC_full\
    .sort_values(by='DeltaV_MC_full')\
    .reset_index(drop=True)

percentile = 5
DeltaV_MC_full_VaR =\
    VaR_percentile(DeltaV_MC_full_sort.iloc[:,0], 
                   percentile)
print("Portfolio 1d VaR at 5% by Monte Carlo (full revaluation) is: ",
      DeltaV_MC_full_VaR)

Portfolio 1d VaR at 5% by Monte Carlo (full revaluation) is:  933041.4898789739


##### <a id = "p42"> </a> 4.2  Monte Carlo - Risk based [back to table of contents](#top)

In [34]:
DeltaV_MC_risk = []
for i in range(len(samples)):
    DeltaV_MC_risk +=\
        [samples.iloc[i,:14]@Para_w]
DeltaV_MC_risk =\
    pd.DataFrame(DeltaV_MC_risk)
DeltaV_MC_risk.columns =\
    ['DeltaV_MC_risk']
DeltaV_MC_risk

,DeltaV_MC_risk
0,1.121940e+06
1,-3.643639e+05
2,1.191352e+05
3,2.593781e+04
4,7.281300e+05
...,...
29995,-3.760489e+05
29996,1.314924e+05
29997,-8.193861e+05
29998,-2.260698e+04


In [35]:
DeltaV_MC_risk_sort =\
    DeltaV_MC_risk\
    .sort_values(by='DeltaV_MC_risk')\
    .reset_index(drop=True)
DeltaV_MC_risk_sort

,DeltaV_MC_risk
0,-2.134246e+06
1,-2.089035e+06
2,-2.082281e+06
3,-2.035031e+06
4,-1.999289e+06
...,...
29995,2.053228e+06
29996,2.063649e+06
29997,2.279979e+06
29998,2.373570e+06


In [36]:
percentile = 5
DeltaV_MC_risk_VaR =\
    VaR_percentile(DeltaV_MC_risk_sort.iloc[:,0], 
                   percentile)
print("Portfolio 1d VaR at 5% by Monte Carlo (risk-based) is: ",
      DeltaV_MC_risk_VaR)

Portfolio 1d VaR at 5% by Monte Carlo (risk-based) is:  927682.8213762073


## <a id = "p5"> Q5. </a>  Historical VaR Model [back to table of contents](#top)

In [37]:
P_risk.head()

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,AAPL,MSFT,F,BAC
2022-11-01 00:00:00,5.209359,4.389513,3.418913,2.963925,2.252118,1.239837,0.286314,-0.429620,-0.972444,-1.437657,-0.017543,-0.017059,0.002244,0.004439
2022-11-02 00:00:00,2.337597,4.739896,4.044957,2.540902,1.527523,1.181667,1.091212,0.986698,0.899499,0.914312,-0.037305,-0.035368,-0.025373,-0.003039
2022-11-03 00:00:00,6.902891,10.979397,10.846471,9.568749,8.321454,7.607816,7.037746,6.239340,5.446595,4.953354,-0.042405,-0.026579,0.015314,-0.005542
2022-11-04 00:00:00,-3.941057,-4.966621,-4.479350,-3.224663,-2.039849,-1.155399,-0.378675,0.454820,1.258218,1.921468,-0.001947,0.033326,0.018854,0.025077
2022-11-07 00:00:00,4.292194,5.361705,4.990763,4.128393,3.318854,3.127938,3.283606,3.417098,3.452884,3.393246,0.003902,0.029270,0.014064,0.005980


##### <a id = "p51"> </a> 5.1  Historical VaR Model - Full Revaluation [back to table of contents](#top)

In [38]:
His_r_samples_full =\
    P_risk.iloc[:, :len(Tenor_list)] *\
    unit_pip +\
    (P_factors.iloc[-1,:len(Tenor_list)]).values

His_s_samples_full =\
    (P_risk.iloc[:, -len(stock_w):]+1) *\
    (P_factors.iloc[-1,-len(stock_w):]).values

His_samples_full =\
    pd.concat([His_r_samples_full, His_s_samples_full],
              axis = 1)
His_samples_full

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,AAPL,MSFT,F,BAC
2022-11-01,0.052766,0.048343,0.045771,0.044641,0.044153,0.043918,0.043808,0.043785,0.043817,0.043879,166.869591,330.884110,9.641585,25.599729
2022-11-02,0.052478,0.048378,0.045834,0.044599,0.044081,0.043912,0.043888,0.043927,0.044005,0.044115,163.512962,324.720832,9.375911,25.409132
2022-11-03,0.052935,0.049002,0.046514,0.045302,0.044760,0.044555,0.044483,0.044452,0.044459,0.044518,162.646744,327.679628,9.767320,25.345346
2022-11-04,0.051851,0.047408,0.044981,0.044022,0.043724,0.043678,0.043741,0.043874,0.044040,0.044215,169.518440,347.845044,9.801373,26.125698
2022-11-07,0.052674,0.048440,0.045928,0.044758,0.044260,0.044107,0.044108,0.044170,0.044260,0.044362,170.511964,346.479732,9.755292,25.638988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.052359,0.048276,0.045846,0.044687,0.044185,0.043959,0.043863,0.043860,0.043915,0.043996,170.281180,337.863618,9.544583,25.386907
2023-10-25,0.052395,0.048297,0.046025,0.045088,0.044822,0.044775,0.044801,0.044880,0.044992,0.045123,167.557663,346.953822,9.746689,25.566635
2023-10-26,0.051835,0.047157,0.044480,0.043290,0.042814,0.042670,0.042677,0.042756,0.042868,0.042995,165.669965,323.998444,9.461613,26.055167
2023-10-27,0.052116,0.047618,0.045069,0.044006,0.043604,0.043537,0.043614,0.043736,0.043878,0.044034,171.202775,338.597925,8.441868,24.559620


In [39]:
DeltaV_His_full = []

for i in range(len(His_samples_full)):
    DF =\
        list(np.exp(-His_samples_full\
                    .iloc[i, :len(T_list)] *\
                    T_list))
    V_payer =\
        V_pay(DF, S, N)
    V_stock =\
        np.array(stock_units) @\
        His_samples_full.iloc[i, -len(stock_units):]
#     print(DF)
#     break
    DeltaV_His_full +=\
        [V_payer+V_stock-Base_V]
    
DeltaV_His_full =\
    pd.DataFrame(DeltaV_His_full)
DeltaV_His_full.columns =\
    ['DeltaV_His_full']
DeltaV_His_full

,DeltaV_His_full
0,-107773.367433
1,-15357.284437
2,381226.868022
3,187047.564794
4,333175.335560
...,...
247,-1018.381280
248,896247.281706
249,-899044.674491
250,-166565.411089


In [40]:
DeltaV_His_full_sort =\
    DeltaV_His_full\
    .sort_values(by='DeltaV_His_full')\
    .reset_index(drop=True)

percentile = 5
DeltaV_His_full_VaR =\
    VaR_percentile(DeltaV_His_full_sort.iloc[:,0], 
                   percentile)
print("Portfolio 1d VaR at 5% by Historical VaR (full revaluation) is: ",
      DeltaV_His_full_VaR)

Portfolio 1d VaR at 5% by Historical VaR (full revaluation) is:  989946.8654037978


##### <a id = "p52"> </a> 5.2  Historical VaR Model - Risk Based [back to table of contents](#top)

In [41]:
DeltaV_His_risk = []
for i in range(len(P_risk)):
    DeltaV_His_risk +=\
        [P_risk.iloc[i,:14]@Para_w]
DeltaV_His_risk =\
    pd.DataFrame(DeltaV_His_risk)
DeltaV_His_risk.columns =\
    ['DeltaV_His_risk']

DeltaV_His_risk_sort =\
    DeltaV_His_risk\
    .sort_values(by='DeltaV_His_risk')\
    .reset_index(drop=True)
DeltaV_His_risk_sort

,DeltaV_His_risk
0,-2.143394e+06
1,-1.958740e+06
2,-1.559743e+06
3,-1.536335e+06
4,-1.344319e+06
...,...
247,1.105768e+06
248,1.120972e+06
249,1.144154e+06
250,1.164659e+06


In [42]:
percentile = 5
DeltaV_His_risk_VaR =\
    VaR_percentile(DeltaV_His_risk_sort.iloc[:,0], 
                   percentile)
print("Portfolio 1d VaR at 5% by Historical VaR (risk-based) is: ",
      DeltaV_His_risk_VaR)

Portfolio 1d VaR at 5% by Historical VaR (risk-based) is:  983865.0263458324


> # Summary

In [43]:
summary =\
    pd.DataFrame([VaR_5,
                  DeltaV_MC_full_VaR,
                  DeltaV_MC_risk_VaR,
                  DeltaV_His_full_VaR,
                  DeltaV_His_risk_VaR])
summary.index =\
    ['Parametric', 
     'Monte Carlo Full Revaluation',  
     'Monte Carlo Risk-based',
     'Historical Full Revaluation', 
     'Historical Risk-based']

summary.columns = ['VaR at 5%']
summary.index.name = f'{num_samples:.1e} samples'
summary.to_csv('QF609 Assignment 1 VaR.csv')
summary

,VaR at 5%
3.0e+04 samples,
Parametric,926596.263348
Monte Carlo Full Revaluation,933041.489879
Monte Carlo Risk-based,927682.821376
Historical Full Revaluation,989946.865404
Historical Risk-based,983865.026346
